# CONTRACTS analyse 
## Instruction:
1. On the left-side panel, change to Files tab 
2. Upload a contract file
3. Right click on the uploaded file -> Copy path
2. Enter (paste) its path/name into the 'Read doc' form below
3. top menu: Runtime/run all

In [0]:
_git_branch = 'master'

#Document parser, refer https://github.com/nemoware/document-parser/releases
lib_version = '1.1.15'

# INIT

## pull code

In [0]:
import json
import subprocess
import sys
import time

import numpy as np
from IPython.core.display import display, HTML
from google.colab import files

!pip install overrides

Ничто = None



def exec(x):
  r = subprocess.check_output(x, shell=True)
  r = r.decode('unicode-escape').encode('latin1').decode('utf8')
  print(r)


print(f"fetching code from GitHub.....{_git_branch}")
try:
  exec('rm -r nlp_tools')
except:
  pass
exec(f'git clone --single-branch --branch {_git_branch} https://github.com/nemoware/analyser.git nlp_tools')

print('🦊 GIT revision:')
exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

sys.path.insert(0, 'nlp_tools')

print('❤️importing Code from GitHub ... DONE')


#----
import matplotlib as mpl
from analyser.documents import TextMap
from analyser.legal_docs import DocumentJson
from colab_support.renderer import HtmlRenderer

 

class DemoRenderer(HtmlRenderer):
  def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    html = self.to_color_text(tokens, weights, colormap, print_debug, _range, separator=separator)
    display(HTML(html))

  def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range, separator=separator)

   
renderer_ = DemoRenderer()

def print_json_summary(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
  print(f'read file {cd.filename}')

  for tag in cd.tags:
    span = tag.span
    _map = cd.tokenization_maps[tag.span_map]
    print(tag)
 
  

In [0]:
!pip install pyjarowinkler

### Init document-parser lib

In [0]:

import os
if not os.path.isfile(f'document-parser-{lib_version}-distribution.zip'):
  !wget https://github.com/nemoware/document-parser/releases/download/$lib_version/document-parser-$lib_version-distribution.zip
if not os.path.isdir(f'document-parser-{lib_version}'):
  !unzip document-parser-$lib_version-distribution.zip

 
os.environ ['documentparser']=f'/content/document-parser-{lib_version}'
from integration.word_document_parser import WordDocParser, join_paragraphs
wp = WordDocParser()

### imports

In [0]:
import os
import pickle
import unittest

import numpy as np

from analyser.contract_parser import ContractAnlysingContext, ContractDocument
from analyser.contract_parser import find_value_sign_currency_attention
 
from analyser.hyperparams import HyperParameters
from analyser.parsing import AuditContext

from analyser.legal_docs import LegalDocument, ContractValue
from analyser.ml_tools import *
from analyser.patterns import *
from analyser.text_tools import *

from tf_support.embedder_elmo import ElmoEmbedder
 

In [0]:
from tf_support.embedder_elmo import ElmoEmbedder
elmo_embedder = ElmoEmbedder()

## 💅 Init Embedder(s)

In [0]:
contract_analyser = ContractAnlysingContext(elmo_embedder)

# Read doc

In [0]:
#@title Enter uploaded file path

filename = '/content/1\u0423\u0441\u043B\u0443\u0433\u0430_\u0414\u043E\u0433\u043E\u0432\u043E\u0440_.doc' #@param {type:"string"}
subsidiary_name = '--\u041E\u043F\u0446\u0438\u043E\u043D\u0430\u043B\u044C\u043D\u043E--' #@param {type:"string"}


results = wp.read_doc(filename)
for doc in results['documents'][:1]:  # XXX
  if 'CONTRACT' == doc['documentType']:    
    doc = join_paragraphs(doc, 'no_id')


for p in doc.paragraphs:
  print ('☢️', p.header.value.strip())

# 🧠 Analyse PHASE 0

In [0]:
# =====================================
from analyser.parsing import AuditContext
actx = AuditContext()
contract_analyser.find_org_date_number(doc, actx)

### render PHASE 0 results

In [0]:
for t in doc.get_tags():
  print(t)
renderer_.render_color_text(doc.tokens,  doc.get_tags_attention() )



# 🧠 Analyse PHASE 1 
(requires phase 0)

In [0]:
contract_analyser.find_attributes(doc, actx)

### render PHASE 1 results

In [0]:
for t in doc.get_tags():
  print('\n', t, '-' * 80)
  print(doc.substr(t))
renderer_.render_color_text(doc.tokens,  doc.get_tags_attention() )

# save to JSON

In [0]:
fn =  f'{filename}.json'
print('saving JSON to', fn)

with open(fn, 'w') as file:
  jjj = DocumentJson(doc)
  file.write(jjj.dumps())
  

In [0]:
doc.to_json_obj()['warnings']

In [0]:
doc.to_json_obj()['attributes']

In [0]:
raise('STOP HERE')